In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import gc
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [2]:
#device = torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cudnn.benchmark = True
cudnn.deterministic = True

In [3]:
def PoissonGen(inp, rescale_fac=2.0):
    rand_inp = torch.rand_like(inp)
    return torch.mul(torch.le(rand_inp * rescale_fac, torch.abs(inp)).float(), torch.sign(inp))

In [4]:
class LIF(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input, th):
        U = input - th
        ctx.save_for_backward(input)
        ctx.th = th
        out = torch.zeros_like(input).cuda()
        out[U > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad = grad_input * 0.3 * F.threshold(1.0 - torch.abs((input-ctx.th)/ctx.th), 0, 0)
        return grad, None

lif = LIF.apply

In [5]:
class VGG_5(nn.Module):
    def __init__(self,time_step,leak,data,th):
        super(VGG_5, self).__init__()
        
        if data == "cifar10":
            input_dim = 3
            pre_linear_dim = 8
        elif data == "mnist":
            input_dim = 1
            pre_linear_dim = 7
            
            
        self.time_step = time_step
        self.th = th
        self.leak = leak
        
        self.s_regs_inp = None
        
        self.conv1 = nn.Conv2d(input_dim, 64, kernel_size=3, padding=1, bias=False)
        self.U_reg_1 = None
        self.S_reg_1 = None
        
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(64,128, kernel_size=3, padding=1, bias=False)
        self.U_reg_2 = None
        self.S_reg_2 = None
        
        self.conv3 = nn.Conv2d(128,128, kernel_size=3, padding=1, bias=False)
        self.U_reg_3 = None
        self.S_reg_3 = None
        
        self.conv4 = nn.Conv2d(128,256, kernel_size=3, padding=1, bias=False)
        self.U_reg_4 = None
        self.S_reg_4 = None
        
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.fc_out = nn.Linear(256*pre_linear_dim*pre_linear_dim, 10, bias=False)
        
        
    def forward(self, inp):

        size = inp.shape

        self.s_regs_inp = torch.zeros(self.time_step,*size, device=device)
        
        u_out = 0
        
        for t in range(self.time_step):
            spike_inp = PoissonGen(inp)
            self.s_regs_inp[t] += spike_inp
            
            x = self.conv1(spike_inp)
            if t == 0:
                self.U_reg_1 = torch.zeros(self.time_step+1,*(x.shape), device=device)
                self.S_reg_1 = torch.zeros(self.time_step+1,*(x.shape), device=device)
            
            U = self.leak*self.U_reg_1[t-1]*(1-self.S_reg_1[t-1]) + x
            x = lif(U, th)
            self.U_reg_1[t] += U
            self.S_reg_1[t] += x

            x = self.pool1(x)
            
            x = self.conv2(x)
            if t == 0:
                self.U_reg_2 = torch.zeros(self.time_step+1,*(x.shape), device=device)
                self.S_reg_2 = torch.zeros(self.time_step+1,*(x.shape), device=device)
            
            U = self.leak*self.U_reg_2[t-1]*(1-self.S_reg_2[t-1]) + x
            x = lif(U, th)
            self.U_reg_2[t] += U
            self.S_reg_2[t] += x
            
            x = self.conv3(x)
            if t == 0:
                self.U_reg_3 = torch.zeros(self.time_step+1,*(x.shape), device=device)
                self.S_reg_3 = torch.zeros(self.time_step+1,*(x.shape), device=device)
            
            U = self.leak*self.U_reg_3[t-1]*(1-self.S_reg_3[t-1]) + x
            x = lif(U, th)
            self.U_reg_3[t] += U
            self.S_reg_3[t] += x
            
            
            x = self.conv4(x)
            if t == 0:
                self.U_reg_4 = torch.zeros(self.time_step+1,*(x.shape), device=device)
                self.S_reg_4 = torch.zeros(self.time_step+1,*(x.shape), device=device)
            
            U = self.leak*self.U_reg_4[t-1]*(1-self.S_reg_4[t-1]) + x
            x = lif(U, th)
            self.U_reg_4[t] += U
            self.S_reg_4[t] += x
                
            x = self.pool2(x)
            
            x = x.view(x.shape[0],-1)
            
            x = self.fc_out(x)
            
            u_out = u_out + x
        return u_out

In [6]:
time_step = 8
leak = 0.95
data = "cifar10"
th = 0.8

model = VGG_5(time_step,leak,data,th).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [7]:
transform_train = transforms.Compose(
    [
     transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
# transform_train = transforms.Compose([transforms.Resize(255), 
#                                        transforms.CenterCrop(224),  
#                                        transforms.RandomHorizontalFlip(),
#                                        transforms.ToTensor(), 
#                                        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                        download=True, transform=transform_train)
train_loader_cifar10 = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                       download=True, transform=transform_test)
test_loader_cifar10 = torch.utils.data.DataLoader(testset, batch_size=batch_size*2,
                                         shuffle=False, num_workers=4, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [8]:
if data == "cifar10":
    train_loader = train_loader_cifar10
    test_loader = test_loader_cifar10
elif data == "mnist":
    train_loader = train_loader_mnist
    test_loader = test_loader_mnist

In [ ]:
for epoch in range(30):
    model.train()
    running_loss = 0.0
    for i, (data, target) in enumerate(train_loader):
        data = data.cuda()
        target = target.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        out = model(data)
        loss = criterion(out, target)
        
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
    
    ## Test each epoch
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

print('Finished Training')

[1,   200] loss: 2.303
[1,   400] loss: 2.268
[1,   600] loss: 2.190
[1,   800] loss: 2.208
[1,  1000] loss: 2.156
[1,  1200] loss: 2.064
[1,  1400] loss: 2.092
[1,  1600] loss: 2.079
[1,  1800] loss: 2.058
[1,  2000] loss: 2.006
[1,  2200] loss: 2.000
[1,  2400] loss: 2.021
[1,  2600] loss: 1.925
[1,  2800] loss: 1.917
[1,  3000] loss: 1.901
[1,  3200] loss: 1.904
[1,  3400] loss: 1.879
[1,  3600] loss: 1.889
[1,  3800] loss: 1.830
[1,  4000] loss: 1.842
[1,  4200] loss: 1.827
[1,  4400] loss: 1.844
[1,  4600] loss: 1.807
[1,  4800] loss: 1.785
[1,  5000] loss: 1.882
[1,  5200] loss: 1.874
